# 2025 Day 2

In [12]:
import os
from getpass import getpass

import aocd
import numpy as np
import pandas as pd
from pydantic import BaseModel

In [2]:
if not os.getenv('AOC_SESSION'):
    os.environ['AOC_SESSION'] = getpass('AOC_SESSION:')

AOC_SESSION: ········


## \--- Day 2: Gift Shop ---

You get inside and take the elevator to its only other stop: the gift shop. "Thank you for visiting the North Pole!" gleefully exclaims a nearby sign. You aren't sure who is even allowed to visit the North Pole, but you know you can access the lobby through here, and from there you can access the rest of the North Pole base.

As you make your way through the surprisingly extensive selection, one of the clerks recognizes you and asks for your help.

As it turns out, one of the younger Elves was playing on a gift shop computer and managed to add a whole bunch of invalid product IDs to their gift shop database! Surely, it would be no trouble for you to identify the invalid product IDs for them, right?

They've even checked most of the product ID ranges already; they only have a few product ID ranges (your puzzle input) that you'll need to check. For example:

```
11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
1698522-1698528,446443-446449,38593856-38593862,565653-565659,
824824821-824824827,2121212118-2121212124
```

(The ID ranges are wrapped here for legibility; in your input, they appear on a single long line.)

The ranges are separated by commas (`,`); each range gives its _first ID_ and _last ID_ separated by a dash (`-`).

Since the young Elf was just doing silly patterns, you can find the _invalid IDs_ by looking for any ID which is made only of some sequence of digits repeated twice. So, `55` (`5` twice), `6464` (`64` twice), and `123123` (`123` twice) would all be invalid IDs.

None of the numbers have leading zeroes; `0101` isn't an ID at all. (`101` is a _valid_ ID that you would ignore.)

Your job is to find all of the invalid IDs that appear in the given ranges. In the above example:

*   `11-22` has two invalid IDs, `_11_` and `_22_`.
*   `95-115` has one invalid ID, `_99_`.
*   `998-1012` has one invalid ID, `_1010_`.
*   `1188511880-1188511890` has one invalid ID, `_1188511885_`.
*   `222220-222224` has one invalid ID, `_222222_`.
*   `1698522-1698528` contains no invalid IDs.
*   `446443-446449` has one invalid ID, `_446446_`.
*   `38593856-38593862` has one invalid ID, `_38593859_`.
*   The rest of the ranges contain no invalid IDs.

Adding up all the invalid IDs in this example produces `_1227775554_`.

_What do you get if you add up all of the invalid IDs?_

In [3]:
puzzle_2 = aocd.models.Puzzle(day=2, year=2025)
puzzle_2.examples[0].input_data

'11-22,95-115,998-1012,1188511880-1188511890,222220-222224,\n1698522-1698528,446443-446449,38593856-38593862,565653-565659,\n824824821-824824827,2121212118-2121212124'

In [43]:
class Range(BaseModel):
    start: int
    end: int
        
    @classmethod
    def new_from_string(cls, start, end):
        return cls(start=start, end=end)


def solve_part_1(puzzle_input: str) -> int:
    ranges: list[Range] = [Range.new_from_string(*map(int, r.strip().split('-', 1))) for r in puzzle_input.split(',')]
    invalid_ids: list[int] = []
    for r in ranges:
        digits_low = len(str(r.start))
        digits_high = len(str(r.end))
        # print(f"digits_low: {digits_low}, digits_high: {digits_high}")
        for digits in range(digits_low, digits_high + 1):
            if digits % 2 == 0:
                # create a sequence of numbers then "concat" to make the repeating pattern
                half_digit = digits // 2
                sequence: list[int] = [(d * 10**half_digit) + d for d in range(1, 10**half_digit)]
                # remove any numbers outside the range
                sequence = [d for d in sequence if d >= r.start and d <= r.end]
                # ensure all the digit lengths are as exepected.
                sequence = [d for d in sequence if len(str(d)) == digits]
                invalid_ids.extend(sequence)
    return sum(invalid_ids), invalid_ids

result, invalid_ids = solve_part_1(puzzle_2.examples[0].input_data)
result, invalid_ids

(1227775554, [11, 22, 99, 1010, 1188511885, 222222, 446446, 38593859])

In [44]:
result, invalid_ids = solve_part_1(puzzle_2.input_data)
result

40055209690

# --- Part Two ---

The clerk quickly discovers that there are still invalid IDs in the ranges in your list. Maybe the young Elf was doing other silly patterns as well?

Now, an ID is invalid if it is made only of some sequence of digits repeated at least twice. So, 12341234 (1234 two times), 123123123 (123 three times), 1212121212 (12 five times), and 1111111 (1 seven times) are all invalid IDs.

From the same example as before:

```
- 11-22 still has two invalid IDs, 11 and 22.
- 95-115 now has two invalid IDs, 99 and 111.
- 998-1012 now has two invalid IDs, 999 and 1010.
- 1188511880-1188511890 still has one invalid ID, 1188511885.
- 222220-222224 still has one invalid ID, 222222.
- 1698522-1698528 still contains no invalid IDs.
- 446443-446449 still has one invalid ID, 446446.
- 38593856-38593862 still has one invalid ID, 38593859.
- 565653-565659 now has one invalid ID, 565656.
- 824824821-824824827 now has one invalid ID, 824824824.
- 2121212118-2121212124 now has one invalid ID, 2121212121.
```

Adding up all the invalid IDs in this example produces 4174379265.

In [54]:
str(5)*3

'555'

In [61]:
class Range(BaseModel):
    start: int
    end: int

    def get_invalid_ids(self) -> set[int]:
        """Return a set of integer ids that satisfy the "invalid" definition.

        Since a pattern can repeat any number of times, we consider sequences of
        length that match any of the factors of then length of the number.
        """
        digits_start: int = len(str(self.start))
        digits_end: int = len(str(self.end))
        invalid_ids: set[int] = set()
        for digits in range(digits_start, digits_end + 1):
            sequence_digits: list[int] = [i for i in range(1, digits) if digits % i == 0]
            # print(self, digits, sequence_digits)
            for sequence_digit in sequence_digits:
                # Repeat the sequence of length `sequence_digit` `n` times. Filter for numbers in the range.
                n: int = digits // sequence_digit
                sequence: list[int] = [int(str(d)*n) for d in range(1, 10**sequence_digit) if int(str(d)*n) >= self.start and int(str(d)*n) <= self.end]
                # ensure all the digit lengths are as exepected.
                sequence = [d for d in sequence if len(str(d)) == digits]
                # print(f'sequence: {sequence}')
                invalid_ids.update(sequence)
        print(f'Range: {self}, invalid_ids: {invalid_ids}')
        return invalid_ids
        
    @classmethod
    def new_from_string(cls, start, end):
        return cls(start=start, end=end)


def solve_part_2(puzzle_input: str) -> int:
    ranges: list[Range] = [Range.new_from_string(*map(int, r.strip().split('-', 1))) for r in puzzle_input.split(',')]
    invalid_ids: list[int] = []
    for r in ranges:
        invalid_ids.extend(r.get_invalid_ids())
    return sum(invalid_ids), invalid_ids


result, invalid_ids = solve_part_2(puzzle_2.examples[0].input_data)
result, invalid_ids

Range: start=11 end=22, invalid_ids: {11, 22}
Range: start=95 end=115, invalid_ids: {99, 111}
Range: start=998 end=1012, invalid_ids: {1010, 999}
Range: start=1188511880 end=1188511890, invalid_ids: {1188511885}
Range: start=222220 end=222224, invalid_ids: {222222}
Range: start=1698522 end=1698528, invalid_ids: set()
Range: start=446443 end=446449, invalid_ids: {446446}
Range: start=38593856 end=38593862, invalid_ids: {38593859}
Range: start=565653 end=565659, invalid_ids: {565656}
Range: start=824824821 end=824824827, invalid_ids: {824824824}
Range: start=2121212118 end=2121212124, invalid_ids: {2121212121}


(4174379265,
 [11,
  22,
  99,
  111,
  1010,
  999,
  1188511885,
  222222,
  446446,
  38593859,
  565656,
  824824824,
  2121212121])

In [62]:
result, invalid_ids = solve_part_2(puzzle_2.input_data)
result

Range: start=288352 end=412983, invalid_ids: {356356, 289289, 378378, 311311, 400400, 333333, 355355, 377377, 310310, 399399, 332332, 354354, 376376, 309309, 398398, 292929, 331331, 404040, 353353, 375375, 308308, 397397, 330330, 352352, 374374, 307307, 396396, 329329, 363636, 351351, 373373, 306306, 395395, 328328, 350350, 372372, 305305, 394394, 327327, 323232, 349349, 371371, 304304, 393393, 326326, 348348, 370370, 303303, 392392, 325325, 393939, 347347, 369369, 302302, 391391, 324324, 346346, 368368, 301301, 390390, 323323, 412412, 353535, 345345, 367367, 300300, 389389, 322322, 411411, 344344, 366366, 299299, 388388, 321321, 410410, 313131, 343343, 365365, 298298, 387387, 320320, 409409, 342342, 364364, 297297, 386386, 319319, 408408, 341341, 383838, 363363, 296296, 385385, 318318, 407407, 340340, 362362, 295295, 384384, 317317, 406406, 343434, 339339, 361361, 294294, 383383, 316316, 405405, 338338, 360360, 293293, 382382, 315315, 404404, 303030, 337337, 359359, 292292, 381381, 31

50857215650